<img src="https://upload.wikimedia.org/wikipedia/commons/2/25/LogoUNED.jpg" width="120" height="120" />

# Cálculo Numérico y Estadística Aplicada


## Grado en Química - UNED 
## PEC adicional


Toda la documentación de la práctica se puede encontrar y descargar en: https://github.com/cefera/CNyEA-Quimica-UNED

Se recomienda leer la parte 1 de los apuntes de la asignatura y ejecutar el cuaderno de Jupyter <code>TestJuguete_CNyEA.ipynb</code> antes de ejecutar este cuaderno.

En la actualidad exiten numerosas librerías para la resolución de numerosos problemas de cálculo numérico. En esta práctica vamos a aprender a utilizar algunas de dichas librerías que se encuentran implementadas en Python. A lo largo del cuaderno se proporcionan enlaces a la documentación de las librerías. Se recomienda consultarlos para la realización de la PEC.

En esta PEC tendrá que rellenar con trozos de código los huecos señalados. Lea y siga las instrucciones que se van proporcionando a lo largo del cuaderno.

----------------------------------------------------------

Para comenzar, lo primero hace falta que introduzca sus datos en la siguiente celda. Para ello, haga doble click en la celda y rellene los campos. Cuando lo haya hecho pulse <kbd>Shift</kbd>+<kbd>Enter</kbd>

> ---
> - **NOMBRE:** 
> - **APELLIDOS:** 
> - **DNI/NIE/No. PASAPORTE:**
> - **No. de alumno de la UNED:**
> - **FECHA (dd/mm/aaaa):** 
> ---

En la siguiente celda, se va a asignar su número de alumno de la UNED a la variable <code>uned</code>, para ello, sustituya los puntos suspensivos <code>...</code> en la siguiente celda por su número de alumno de la UNED y ejecute la celda pulsando pulse <kbd>Shift</kbd>+<kbd>Enter</kbd>.

In [ ]:
uned = ...

## Carga de librerías

La siguiente celda carga todas las librerías y los datos que posteriormente utilizaremos. Ejecútela.

In [ ]:
###########################################################################################
#
#   Librerias
#
###########################################################################################

###########################################################################################
#   Estas son las librerías que se van a emplear en este ejemplo
#   Las librerías son códigos ya desarrollados por otras personas que podemos reutilizar
###########################################################################################

import numpy as np; # esta librería contiene una gran cantidad de funciones de cálculo numérico
import matplotlib.pyplot as plt # esta librería sirve para dibujar gráficas
from scipy import linalg # descomposición LU
from scipy import optimize # para newton y bisección
import scipy.integrate as integrate # integracion

###########################################################
#   Esta es la definición de un par de colores que vamos a utilizar posteriormente en las figuras
jpac_blue   = "#1F77B4"; jpac_red    = "#D61D28"; jpac_green  = "#2CA02C"; jpac_orange = "#FF7F0E";
jpac_purple = "#9467BD"; jpac_brown  = "#8C564B"; jpac_pink   = "#E377C2"; jpac_gold   = "#BCBD22";
jpac_aqua   = "#17BECF"; jpac_grey   = "#7F7F7F";

jpac_color = [jpac_blue, jpac_red, jpac_green, jpac_orange, jpac_purple, jpac_brown,
              jpac_pink, jpac_gold, jpac_aqua, jpac_grey ];

#generacion de los numeros a utilizar
rng = np.random.default_rng(uned);
#print(rng)

# Matrices
def construir_matriz(n,nmin,nmax):
    tamano = int(n*(n+1)/2)
    rints = rng.integers(low=nmin, high=nmax, size=tamano)
    intsg = rng.binomial(1, 0.5, size=tamano)
    intsg = np.where(intsg == 0, -1, intsg)
    A = np.zeros((n,n),dtype='int')
    punto = 0
    for i in np.arange(n):
        A [i,i] = intsg[punto]*rints[punto]
        punto = punto + 1
        for j in np.arange(i+1,n):
            A [i,j] = intsg[punto]*rints[punto]
            A [j,i] = A[i,j]
            punto = punto + 1
    return A

Asize, Amin, Amax = 4, 1, 10; rechazado = True
while (rechazado):
    Ainput = construir_matriz(Asize,Amin,Amax)
    if np.linalg.det(Ainput)!=0:
        rechazado = False

binput = np.transpose(rng.integers(low=Amin, high=Amax, size=Asize))

# Datos para interpolar
def construir_datos(n,xmin,xmax):
    x = np.linspace(xmin,xmax,n)
    y = 10.*rng.random(x.size)
    return x,y

def construir_polinomio(n,nmin,nmax):
    rints = rng.integers(low=nmin, high=nmax, size=n)
    intsg = rng.binomial(1, 0.5, size=n)
    intsg = np.where(intsg == 0, -1, intsg)
    return np.array(intsg*rints)

ndatos = 6
xinput, yinput = construir_datos(ndatos,1.,11.)

# Polinomio para resolución de ecuaciones no lineales
qroots = rng.integers(low=-10, high=10, size=3)
qpol = np.polynomial.polynomial.Polynomial.fromroots(qroots)
qcoef = np.flip(qpol.coef)
qpol = np.poly1d(qcoef)
       
print('Ahora las librerías y los datos están cargados y disponibles para ser utilizados')

Hemos cargado cinco librerías que utilizaremos a lo largo de la PEC: <code>numpy</code> con el alias <code>np</code>, <code>matplotlib.pyplot</code> con el alias <code>plt</code>, <code>scipy.linalg</code> con el alias <code>linalg</code>, <code>scipy.optimize</code> con el alias <code>optimize</code> y <code>scipy.integrate</code> con el alias <code>integrate</code>

En primer lugar vamos a realizar un cálculo sencillo. El coseno de $\pi$. Para ello vamos a utilizar la función <code>cos</code> de <code>numpy</code> y el número $\pi$ que está almacenado en la variable <code>pi</code> de la misma libreria. Por ello, si queremos calcular $\cos \pi$ la línea de código a escribir será <code>np.cos(np.pi)</code>. Sustituya los puntos suspensivos <code>...</code> en la siguiente celda por <code>np.cos(np.pi)</code> y ejecútela.

In [ ]:
...

**En el resto de la práctica se le solicitará que sustituya conjuntos de puntos suspensivos <code>...</code> por el código apropiado tal y como acaba de hacer.**

Vamos a comenzar con la resolución de sistemas de ecuaciones.

## Sistemas de ecuaciones

Ejecute la siguiente celda.

In [ ]:
print('Primero se va a trabajar con sistemas de ecuaciones.')
print('Supongamos un sistema de ecuaciones lineales dado por la siguiente matriz A simétrica:')
print('A=',Ainput)
print('Y por el siguiente vector b:')
for i in range(Asize):
    if i==0:
        print('b=[',binput[i])
    elif i==Asize-1:
        print('   ',binput[i],']')
    else:
        print('   ',binput[i])

Vamos a calcular la factorización $LU$ y a diagonalizar la matriz $A$

En primer lugar defina la matriz $A$ y el vector $b$. Para ello ha de utilizar la función <code>array</code> de <code>numpy</code> cuya documentación puede encontrar en https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array. Rellene los huecos <code>...</code> de la siguiente celda con los valores de $A$ y $b$ dados en la celda anterior.

In [ ]:
A = np.array([ ... ] );
b = np.array([ ... ] );
print('Comprobamos que A está correctamente definido imprimiendolo en pantalla')
print('A =',A)
print('Comprobamos que b está correctamente definido imprimiendolo en pantalla (aparece transpuesto, eso está bien)')
print('b =',b)

#### Factorización $LU$

Para calcular la factorización LU sólo hay que utilizar la función de <code>scipy.linalg.lu</code> documentada en https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html En concreto el código a escribir sería:

<code>P, L, U = linalg.lu(A)</code>

Nótese que se escribe <code>linalg.lu</code> para invocar la función en lugar de <code>scipy.linalg.lu</code>. Piense por qué eso es así. Entenderlo le será muy útil en lo que resta de PEC.

Calcule la descomposición LU de la matriz A en la siguiente celda e imprima en pantalla los resultados:

In [ ]:
P, L, U = ...
print(P)
print(L)
print(U)

A continuación compruebe que la factorización es correcta y se cumple:
$$L U = A$$

Para ello, puede hacer uso de la función de multiplicación de matrices de <code>numpy</code> llamada <code>matmul</code> https://numpy.org/doc/stable/reference/generated/numpy.matmul.html Imprima el resultado en pantalla

In [ ]:
A = ...
print(A)

La factorización $LU$ se puede explotar para resolver el sistema
$$Ax=b$$
Sin embargo, es más directo resolver el sistema utilizando la librería <code>solve</code> de <code>scipy.linalg</code> https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html. Hágalo en la siguiente celda

In [ ]:
X = ...
print(X)

#### Diagonalización

A continuación realice la diagonalización de $A$ calculando los autovalores y los autovectores usando la librería <code>eig</code> de <code>numpy</code> https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html

In [ ]:
w, v = ...
print('Autovalores= ',w)
print('Autovectores= ',v)

## Interpolación

En el cuaderno <code>TestJuguete_CNyEA.ipynb</code> se proporciona el código para reproducir la Figura 3.2 de los apuntes (ejecute la siguiente celda):

In [ ]:
print('Copie y pegue dicho código en la siguiente celda y modifíquelo para')
print('obtener los resultados de la Figura 3.2 con los siguientes')
print('pares de puntos (x,y):')
for i in range(ndatos):
    print('(',xinput[i],',',yinput[i],')')
print('con un polinomio de grado:',yinput.size-1)
print('y calcule el valor en x=6')
print('calcule también el valor en x=6 si se interpola linealmente entre los puntos x=5 y x=7')


In [ ]:
# sustituya los puntos suspensivos por el código adecuado

...

## Resolución de ecuaciones no lineales

El paso siguiente es resolver una ecuación no lineal aplicando el método de la bisección. La ecuación elegida es:
$$q(x)=0$$
donde $q(x)$ es un polinomio que fue generado al principio del cuaderno y cuyos coeficientes están almacenados en  <code>qpol</code>. Para calcular las raices de un polinomio, <code>numpy</code> tiene la función <code>roots</code> https://numpy.org/doc/stable/reference/generated/numpy.roots.html#numpy.roots Calcule en la siguiente celda las raices de dicho polinomio utilizando este procedimiento.

In [ ]:
...

La solución han de ser raices reales. Esta función <code>roots</code> se puede utilizar porque queremos las raices de un polinomio, pero nos interesa resolver casos más generales y para ello vamos a aprender a usar los métodos de la bisección y de Newton, lo cuál es inmediato utilizando las funciones <code>bisect</code> https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html#scipy.optimize.bisect de la librería <code>scipy.optimize</code> que hemos cargado con el alias <code>optimize</code>. Para obtener una raiz del polinomio anterior utilizando el método de la bisección, ejecute la siguiente celda:

In [ ]:
def f(x):
    return np.polyval(qpol,x);

raiz = optimize.bisect(f, -10.,10.)
print(raiz)

Donde <code>f</code> es la función (el polinomio en este caso) del que queremos obtener la raiz, y  <code>-10.,10.</code> definen el rango en el que vamos a buscar la raiz. Este método sólo proporciona una raiz. Calcule la(s) demás en la siguiente celda utilizando el mismo método:

In [ ]:
raiz = ...
print(raiz)
raiz = ...
print(raiz)

## Derivación e integración numérica

Vamos a derivar y a integrar numéricamente el polinomio que hemos utilizado en el ejercicio de interpolación. Para ello vamos a emplear las funciones <code>gradient</code> de <code>numpy</code> para derivar https://numpy.org/doc/stable/reference/generated/numpy.gradient.html y <code>scipy.integrate</code> cargada con el alias <code>integrate</code> para integrar https://docs.scipy.org/doc/scipy/tutorial/integrate.html.

### Derivación

Rellene los huecos <code>...</code> y ejecute la celda

In [ ]:
x_para_gradiente = np.linspace(1.,10.,100)
y_para_gradiente = np.array([ ... for i in x_para_gradiente])
derivada = ...

fig = plt.figure(figsize=(5, 5))
plt.plot(x_para_gradiente,y_para_gradiente,c=jpac_color[3],label='Funcion')
plt.plot(x_para_gradiente,derivada,'--',c=jpac_color[1],label='Derivada')
plt.legend(loc='upper left')
plt.show()

### Integración

Primero hay que definir el integrando. En nuestro caso el polinomio $p(x)$.

In [ ]:
def integrando(x):
    return np.polyval(qpol,x);

Y para integrar vamos a utilizar la función <code>integrate.quad</code> para integrar $p(x)$ entre 5 y 7, la cuál proporciona dos valores, el resultado de la integral y el error numérico. Complete la siguiente celda rellenado los huecos <code>...</code> y ejecute:

In [ ]:
I, error = ...
print('Valor de la integral=',I)
print('Error numérico=',error)

**Con esto se termina la PEC adicional de la asignatura.**

**Proceda a imprimirla. En Binder vaya al menú File arriba a la izquierda y seleccione Print. Se debería generar un pdf con toda la práctica. A continuación, suba el pdf antes del XXXX al apartado de tareas con el epígrafe "XXXXXX" del aLF**